In [3]:
import pandas as pd
import re
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
import nltk
from nltk.corpus import stopwords
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle
from gensim.models import LdaModel
from gensim.corpora import Dictionary
# import pyLDAvis.gensim
import warnings
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import numpy as np
import pandas as pd
import string
import re
from tqdm._tqdm_notebook import tqdm_notebook,tnrange,tqdm
from collections import Counter,OrderedDict
from gensim import models,corpora
# from gensim.summarization import summarize,keywords
import warnings
# import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
# import pyLDAvis.gensim
import gensim.models.phrases as gen
from gensim.models.coherencemodel import CoherenceModel

In [8]:
import gdown

In [9]:
id = 'https://drive.google.com/uc?id=1b3rmCSIoh6VhD4HKWjI4HOW-cSwcwbeC'
output = 'arxiv-dataset.zip'

gdown.download(id, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1b3rmCSIoh6VhD4HKWjI4HOW-cSwcwbeC
From (redirected): https://drive.google.com/uc?id=1b3rmCSIoh6VhD4HKWjI4HOW-cSwcwbeC&confirm=t&uuid=826c7198-c9a9-4ce5-acb5-7d7cb24419ed
To: /content/arxiv-dataset.zip
100%|██████████| 3.62G/3.62G [00:34<00:00, 105MB/s]


'arxiv-dataset.zip'

In [10]:
!unzip arxiv-dataset.zip

Archive:  arxiv-dataset.zip
   creating: arxiv-dataset/
  inflating: __MACOSX/._arxiv-dataset  
  inflating: arxiv-dataset/train.txt  
  inflating: __MACOSX/arxiv-dataset/._train.txt  
  inflating: arxiv-dataset/vocab     
  inflating: __MACOSX/arxiv-dataset/._vocab  
  inflating: arxiv-dataset/test.txt  
  inflating: __MACOSX/arxiv-dataset/._test.txt  
  inflating: arxiv-dataset/val.txt   
  inflating: __MACOSX/arxiv-dataset/._val.txt  


In [11]:
train_file_path = "/content/arxiv-dataset/train.txt"
test_file_path = "/content/arxiv-dataset/test.txt"
val_file_path = "/content/arxiv-dataset/val.txt"
vocab_file_path = "/content/arxiv-dataset/vocab"

In [12]:
import json
import pandas as pd

In [13]:
def read_json_instances_up_to_limit(file_path, limit=25000):
    data = []
    with open(file_path, 'r') as file:
        instance_count = 0
        while instance_count < limit:
            line = file.readline()
            if not line:
                break  # End of file
            instance = json.loads(line)
            data.append(instance)
            instance_count += 1
    return data

# Read exactly 25,000 training instances


In [31]:
train_data = read_json_instances_up_to_limit(train_file_path, limit=15000)

In [32]:
train_df = pd.DataFrame(train_data)

In [ ]:
del train_data

In [15]:
test_data = read_json_instances_up_to_limit(test_file_path,limit=10000)

In [16]:
test_df = pd.DataFrame(test_data)

In [19]:
test_df.head()

,article_id,article_text,abstract_text,labels,section_names,sections
0,1009.3123,for about 20 years the problem of properties o...,<S> the short - term periodicities of the dail...,None,"[introduction, methods of periodicity analysis...",[[for about 20 years the problem of properties...
1,1512.09139,it is believed that the direct detection of gr...,<S> we study the detectability of circular pol...,None,"[introduction, stokes parameters for plane gra...",[[it is believed that the direct detection of ...
2,0909.1602,"as a common quantum phenomenon , the tunneling...","<S> starting from the wkb approximation , a ne...",None,"[[sec:intro]introduction, [sec:formalism]forma...","[[as a common quantum phenomenon , the tunneli..."
3,1512.03812,for the hybrid monte carlo algorithm ( hmc)@xc...,<S> we study a novel class of numerical integr...,None,"[introduction, geometric integrators for hmc, ...",[[for the hybrid monte carlo algorithm ( hmc)@...
4,1512.09024,recently it was discovered that feynman integr...,<S> new methods for obtaining functional equat...,None,"[introduction, deriving functional equations f...",[[recently it was discovered that feynman inte...


In [33]:
import pandas as pd

# Assuming train_df is your DataFrame
for index, row in train_df.iterrows():
    concatenated_article_text = ' '.join(row['article_text']) if isinstance(row['article_text'], list) else row['article_text']
    concatenated_abstract_text = ' '.join(row['abstract_text']) if isinstance(row['abstract_text'], list) else row['abstract_text']
    train_df.at[index, 'article_text'] = concatenated_article_text
    train_df.at[index, 'abstract_text'] = concatenated_abstract_text


In [18]:
import pandas as pd

# Assuming train_df is your DataFrame
for index, row in test_df.iterrows():
    concatenated_article_text = ' '.join(row['article_text']) if isinstance(row['article_text'], list) else row['article_text']
    concatenated_abstract_text = ' '.join(row['abstract_text']) if isinstance(row['abstract_text'], list) else row['abstract_text']
    test_df.at[index, 'article_text'] = concatenated_article_text
    test_df.at[index, 'abstract_text'] = concatenated_abstract_text


In [38]:
data=train_df
len(data)

15000

In [ ]:
train_df.head()

,article_id,article_text,abstract_text,labels,section_names,sections
0,1405.3379,additive models @xcite provide an important fa...,<S> additive models play an important role in ...,None,"[introduction, main results on learning rates,...",[[additive models @xcite provide an important ...
1,0901.1147,the leptonic decays of a charged pseudoscalar ...,<S> we have studied the leptonic decay @xmath0...,None,"[[sec:introduction]introduction, [sec:detector...",[[the leptonic decays of a charged pseudoscala...
2,nlin0608019,the transport properties of nonlinear non - eq...,"<S> in 84 , 258 ( 2000 ) , mateos conjectured ...",None,"[introduction, regularity and chaos in single-...",[[the transport properties of nonlinear non - ...
3,0903.5449,studies of laser beams propagating through tur...,<S> the effect of a random phase diffuser on f...,None,"[introduction, the method of photon distributi...",[[studies of laser beams propagating through t...
4,hep-ph0605279,the so - called `` nucleon spin crisis '' rais...,<S> with a special intention of clarifying the...,None,"[introduction, model lagrangian with pion mass...",[[the so - called `` nucleon spin crisis '' ra...


In [ ]:
data.head()

,article_id,article_text,abstract_text,labels,section_names,sections
0,1405.3379,additive models @xcite provide an important fa...,<S> additive models play an important role in ...,None,"[introduction, main results on learning rates,...",[[additive models @xcite provide an important ...
1,0901.1147,the leptonic decays of a charged pseudoscalar ...,<S> we have studied the leptonic decay @xmath0...,None,"[[sec:introduction]introduction, [sec:detector...",[[the leptonic decays of a charged pseudoscala...
2,nlin0608019,the transport properties of nonlinear non - eq...,"<S> in 84 , 258 ( 2000 ) , mateos conjectured ...",None,"[introduction, regularity and chaos in single-...",[[the transport properties of nonlinear non - ...
3,0903.5449,studies of laser beams propagating through tur...,<S> the effect of a random phase diffuser on f...,None,"[introduction, the method of photon distributi...",[[studies of laser beams propagating through t...
4,hep-ph0605279,the so - called `` nucleon spin crisis '' rais...,<S> with a special intention of clarifying the...,None,"[introduction, model lagrangian with pion mass...",[[the so - called `` nucleon spin crisis '' ra...


In [39]:
docs = [x for x in data['article_text']]

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['xcite', 'fig', 'eq', 'xmath','et','al','following','proof','let','theorm','mu','delta','gamma','lambda','omega','pi','sigma','data','research','result','beta'])

# Data Preprocessing
We use NLTK’s Wordnet to find the meanings of words, synonyms, antonyms, and more. In addition, we use WordNetLemmatizer to get the root word. Filter out stop words.

In [20]:
# A function to prepare the text for topic modelling
def words(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    text = regex.sub(" ", text.lower())
    words = text.split()
    words = [re.sub('\S*@\S*\s?', '', sent) for sent in words]
    words = [re.sub('\s+', ' ', sent) for sent in words]
    words = [re.sub("\'", "", sent) for sent in words]
    words = [w for w in words if not len(w) < 2]
    words = [w for w in words if w not in stop_words]
    words = [lmtzr.lemmatize(w) for w in words]
    words = [w for w in words if not w.startswith('xmath')]  # Filter out words starting with 'xmath'

    return words

In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [40]:
docs = [words(x) for x in data['article_text']]

In [41]:
dictionary = Dictionary(docs)

In [42]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Topic Modeling using LDA
LDA :: Latent Dirichlet Allocation is a probabilistic model. It is a unsupervised machine learning technique. And to obtain cluster assignments, it uses two probability values: P( word | topics) and P( topics | documents).

pyLDAvis is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

The size of the bubble measures the importance of the topics, relative to the data.
When we have 5 or 10 topics, we can see certain topics are clustered together (overlapping bubbles), this indicates the similarity between topics.

In [36]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [44]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=50,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=500,
                                           passes=5,
                                           alpha='auto',
                                           per_word_topics=True)

In [47]:
get_lda_topics(lda_model, 30)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,line,potential,mass,ray,para,hole,surface,right,let,galaxy,...,spin,channel,model,data,pt,field,mass,electron,one,temperature
1,flux,atom,galaxy,emission,delany,black,force,frac,theorem,line,...,state,link,neutrino,distribution,bf,magnetic,hep,energy,algorithm,phase
2,current,molecule,cluster,energy,jafelice,metric,particle,left,function,al,...,lattice,rate,energy,model,em,velocity,phys,band,problem,system
3,height,vortex,star,source,didacci,solution,cell,end,set,et,...,site,noop,matter,value,rule,density,quark,state,time,transition
4,region,interaction,gas,spectrum,gbhard,horizon,simulation,aligned,lemma,source,...,two,code,universe,function,ex,rotation,decay,structure,set,density
5,width,energy,model,al,xmb,gravitational,length,nonumber,since,emission,...,hamiltonian,doibase,scale,error,ensuremath,flow,higgs,temperature,method,model
6,figure,soliton,disk,et,tseh,gravity,interface,alpha,proof,image,...,model,power,dark,parameter,formula,direction,arxiv,calculation,number,value
7,eps,atomic,formation,model,somegam,coordinate,wall,mu,following,region,...,symmetry,http,density,method,put,plasma,model,result,step,order
8,solar,state,density,kev,smallangle,curvature,pressure,delta,follows,redshift,...,ground,user,cosmological,result,mathsf,electric,ph,effect,first,energy
9,layer,reaction,halo,time,setcn,spacetime,structure,lambda,case,dust,...,phase,org,spectrum,using,program,scale,coupling,charge,example,function


The above is the visual representation of the topic modelling and the below is the raw representation and we can specify #words with highest probability to display in each topic.

In [ ]:
lda_model.save('model10.gensim')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
lda_model.save('/content/drive/MyDrive/Models/tm/model20.gensim')

In [4]:
lda_model = LdaModel.load('/content/drive/MyDrive/Models/tm/model10.gensim')

In [ ]:
# lda_model.save('/content/drive/MyDrive/Models/tm/model10.gensim')
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

In [63]:
single_text = words(test_df['article_text'][100])
# tokenized_text = simple_preprocess(single_text)

# Create a dictionary from the tokenized text
dictionary = Dictionary([single_text])

# Convert the tokenized text into a bag-of-words representation
bow_vector = dictionary.doc2bow(single_text)

# Use the trained model for inference
topic_distribution = lda_model.get_document_topics(bow_vector)

In [ ]:
test_df['article_text'][100]

In [ ]:
topic_distribution

In [ ]:
for topic_id, topic_prob in topic_distribution:
    print("Topic:", topic_id)
    print("Top Words:", lda_model.show_topic(topic_id,topn=20))